In [1]:
import json
from tensorflow.keras.models import Sequential, model_from_json
with open("F:\\SER DCA\\DCA_SER.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("F:\\SER DCA\\DCA_SER.weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [2]:
import librosa
import numpy as np 

def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])
    
    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result


In [3]:
from sklearn.preprocessing import OneHotEncoder
classes = ['disgust', 'sad', 'fear', 'happy', 'angry', 'neutral', 'surprise']
encoder = OneHotEncoder()
encoder.fit_transform(np.array(classes).reshape(-1,1))

<7x7 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [6]:
import pandas as  pd
def predict_emotion(audio_chunk):
    # Extract features from audio chunk
    features = np.array(extract_features(audio_chunk))
    prde = pd.DataFrame(features)
    prde.shape
    new_data = np.zeros((1, 4158))
    new_data[:, :prde.shape[0]] = prde.T
    prde1 = pd.DataFrame(new_data)
    prde1 = prde1.fillna(0)
    # Reshape features for model input
    prde1 = np.expand_dims(prde1, axis=2)
    # Predict emotion using the model
    emotions = loaded_model.predict(prde1)
    predictions = encoder.inverse_transform(emotions)
    return predictions

In [15]:
# Load longer audio file
audio_file_path = "C:\\Users\\shreeram kulkarni\\Downloads\\Prodigal Son_ Someone Saw Me.wav"
audio, sample_rate = librosa.load(audio_file_path)

# Define chunk size and overlap
chunk_size = 4.38 # in seconds
overlap = 0  # in seconds

# Process audio in chunks
predictions = []
start = 0
end = int(chunk_size * sample_rate)
while end <= len(audio):
    # Extract audio chunk
    audio_chunk = audio[start:end]
    # Predict emotion for chunk
    emotion_probabilities = predict_emotion(audio_chunk)
    # Record prediction
    predictions.append(emotion_probabilities[0][0])
    # Move to next chunk
    start += int(chunk_size * sample_rate) - int(overlap * sample_rate)
    end = start + int(chunk_size * sample_rate)

# Combine predictions (e.g., average probabilities)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
['neutral', 'disgust', 'sad', 'angry', 'neutral', 'disgust', 'sad', 'neutral', 'sad', 'sad', 'disgust', 'sad', 'neutral', 'angry', 'angry', 'neutral', 'happy', 'sad', 'neutral', 'neutral']


In [20]:
import collections
import tabulate

emotion_counts = collections.Counter(predictions)
total_observations = len(predictions)
emotion_percentages = {emotion: count/total_observations*100 for emotion, count in emotion_counts.items()}

headers = ['Emotion', 'Percentage']
table = [[emotion, f"{percentage:.2f}%"] for emotion, percentage in emotion_percentages.items()]
table.insert(0, headers)

print(tabulate.tabulate(table, headers='firstrow'))

Emotion    Percentage
---------  ------------
neutral    35.00%
disgust    15.00%
sad        30.00%
angry      15.00%
happy      5.00%


In [18]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.
